### Funds of funds

In [1]:
# Input
date = '2018_09_21'
homePath = 'C:/Users/Mamed/Python4DS/'
projPath = homePath + 'FundsBR/'
strucPath = projPath + 'Structures_' + date + '/'
libsPath = projPath + 'Libs/'

In [2]:
import pandas as pd
import numpy as np
from collections import Counter
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import gridspec
from IPython.display import clear_output
import copy

# Load my libraries
exec(open(libsPath + 'fundsLib.py').read())

#### Funds of funds (blk 2)

In [4]:
# Read BLC
dfBLC2 = pd.read_pickle(strucPath + 'BLC_blk_2_.pkl')
l0 = dfBLC2.shape[0]
print('{:>12,.0f} rows {:>6,.0f} columns'.format(dfBLC2.shape[0], dfBLC2.shape[1]))

   7,865,148 rows     18 columns


In [5]:
# Summary
dfSummary(dfBLC2)

RangeIndex: 7865148 entries, 0 to 7865147
Data columns (total 18 columns):

                    colType   Unique      NaN
TP_FUNDO             object       13        0
CNPJ_FUNDO           object    24747        0
DENOM_SOCIAL         object    24812        0
DT_COMPTC            object      164        0
TP_APLIC             object        4        0
TP_ATIVO             object        7    81352
EMISSOR_LIGADO       object        2   681143
TP_NEGOC             object        2  7184005
QT_VENDA_NEGOC      float64   117419  5026818
VL_VENDA_NEGOC      float64    98922  5000456
QT_AQUIS_NEGOC      float64    67480  5027372
VL_AQUIS_NEGOC      float64    48469  5001010
QT_POS_FINAL        float64  2675576    81352
VL_MERC_POS_FINAL   float64  7176443        0
VL_CUSTO_POS_FINAL  float64   365929  7184005
DT_CONFID_APLIC      object      807  4934528
CNPJ_FUNDO_COTA      object    12273    81352
NM_FUNDO_COTA        object    12297    81352


In [11]:
# The confidentiality period (last 3 months) will have null 'CNPJ_FUNDO_COTA',
# i.e., for that dates, we will not be able to see the invested funds.
# Therefore, we delete those rows.
dfBLC2 = dfBLC2[~dfBLC2['CNPJ_FUNDO_COTA'].apply(pd.isnull)].reset_index(drop = True)
l1 = dfBLC2.shape[0]
print('Deleted rows: {:<20,.0f}'.format(l0 - l1))

Deleted rows: 81,352              


In [13]:
# Two date columns have to be convervet from string to datestamp
dfBLC2['DT_COMPTC'] = pd.to_datetime(dfBLC2['DT_COMPTC'])
dfBLC2['DT_CONFID_APLIC'] = pd.to_datetime(dfBLC2['DT_CONFID_APLIC'])

In [14]:
# Summary
dfSummary(dfBLC2)

RangeIndex: 7783796 entries, 0 to 7783795
Data columns (total 18 columns):

                           colType   Unique      NaN
TP_FUNDO                    object       13        0
CNPJ_FUNDO                  object    24651        0
DENOM_SOCIAL                object    24716        0
DT_COMPTC           datetime64[ns]      164        0
TP_APLIC                    object        4        0
TP_ATIVO                    object        7        0
EMISSOR_LIGADO              object        2   681117
TP_NEGOC                    object        2  7102679
QT_VENDA_NEGOC             float64   117419  4945466
VL_VENDA_NEGOC             float64    98587  4945466
QT_AQUIS_NEGOC             float64    67480  4946020
VL_AQUIS_NEGOC             float64    48227  4946020
QT_POS_FINAL               float64  2675576        0
VL_MERC_POS_FINAL          float64  7104683        0
VL_CUSTO_POS_FINAL         float64   365908  7102679
DT_CONFID_APLIC     datetime64[ns]      803  4934528
CNPJ_FUNDO_COTA        

In [6]:
# Fund type
seriesSummary(dfBLC2, 'TP_FUNDO', 0)

FI           7827490
FACFIF         22624
FIP             6986
FIF             6305
FITVM            672
FIC-FITVM        423
FMIEE            272
FAPI             225
FMP-FGTS          61
FIIM              54
FMAI              23
FIFDIV            12
FUNCINE            1
Name: TP_FUNDO, dtype: int64

Type:  object
Unique:  13
NaNs:  0


In [7]:
# Investment type
seriesSummary(dfBLC2, 'TP_APLIC', 5)

Cotas de Fundos                                       7184005
Cotas de fundos de investimento - Instrução Nº 409     620717
Cotas de fundos de renda fixa                           31196
Cotas de fundos de renda variável                       29230
Name: TP_APLIC, dtype: int64

Type:  object
Unique:  4
NaNs:  0


In [8]:
# Investment type
seriesSummary(dfBLC2, 'TP_ATIVO', 0)

Fundo de Investimento e de Cotas    6498135
Cotas de fundos                      681117
FIDC                                 495896
NaN                                   81352
FI Participações                      73267
FI Imobiliário                        19354
Outros tipos de fundo                 15379
FMI Empresas Emergentes                 648
Name: TP_ATIVO, dtype: int64

Type:  object
Unique:  8
NaNs:  81352


 ### Temos uma rede de investimentos em cascata até chegar no ativo financeiro final. Ou seja, essa rede não adiciona informação quanto à alocação final nos ativos financeiros, uma vez que bastaria para isso olhar somente o ultimo fundo da cadeia, aquele que realmente faz as alocações.
 
### Portanto, não vamos incluir estes em nosso estudo.

In [29]:
dfBLC2[(dfBLC2['CNPJ_FUNDO_COTA'] == '18.391.171/0001-54') & 
       (dfBLC2['DT_COMPTC'] == '2015-08-31')]

,TP_FUNDO,CNPJ_FUNDO,DENOM_SOCIAL,DT_COMPTC,TP_APLIC,TP_ATIVO,EMISSOR_LIGADO,TP_NEGOC,QT_VENDA_NEGOC,VL_VENDA_NEGOC,QT_AQUIS_NEGOC,VL_AQUIS_NEGOC,QT_POS_FINAL,VL_MERC_POS_FINAL,VL_CUSTO_POS_FINAL,DT_CONFID_APLIC,CNPJ_FUNDO_COTA,NM_FUNDO_COTA
5100000,FI,08.537.800/0001-58,SANTANDER FIC FI PB CANOE MULTIMERCADO CREDITO...,2015-08-31,Cotas de Fundos,Fundo de Investimento e de Cotas,N,NaN,NaN,NaN,NaN,NaN,5587.430980,900296.78,NaN,NaT,18.391.171/0001-54,GÁVEA MACRO DÓLAR FUNDO DE INVESTIMENTO EM COT...
5159459,FI,09.241.797/0001-93,BLESS FUNDO DE INVESTIMENTO MULTIMERCADO CREDI...,2015-08-31,Cotas de Fundos,Fundo de Investimento e de Cotas,S,NaN,NaN,NaN,NaN,NaN,13324.945608,2147034.24,NaN,NaT,18.391.171/0001-54,GÁVEA MACRO DÓLAR FUNDO DE INVESTIMENTO EM COT...
5223956,FI,10.786.565/0001-08,ROMA 1947 FUNDO DE INVESTIMENTO MULTIMERCADO C...,2015-08-31,Cotas de Fundos,Fundo de Investimento e de Cotas,S,NaN,NaN,NaN,NaN,NaN,50661.472150,8163028.85,NaN,2015-11-29,18.391.171/0001-54,GÁVEA MACRO DÓLAR FUNDO DE INVESTIMENTO EM COT...
5318175,FI,12.765.674/0001-66,ABERDEEN FUNDO DE INVESTIMENTO MULTIMERCADO CR...,2015-08-31,Cotas de Fundos,Fundo de Investimento e de Cotas,N,NaN,NaN,NaN,NaN,NaN,7712.000000,1242593.45,NaN,2015-11-29,18.391.171/0001-54,GÁVEA MACRO DÓLAR FUNDO DE INVESTIMENTO EM COT...
5443367,FI,15.675.250/0001-07,RAMP + 8 FUNDO DE INVESTIMENTO MULTIMERCADO CR...,2015-08-31,Cotas de Fundos,Fundo de Investimento e de Cotas,S,NaN,NaN,NaN,NaN,NaN,5049.176760,813568.45,NaN,NaT,18.391.171/0001-54,GÁVEA MACRO DÓLAR FUNDO DE INVESTIMENTO EM COT...
5498613,FI,17.340.475/0001-20,FEEDER GAVEA MACRO DOLAR FUNDO DE INVESTIMENTO...,2015-08-31,Cotas de Fundos,Fundo de Investimento e de Cotas,N,NaN,0.0,0.0,0.0,0.0,561987.098270,90552380.38,NaN,NaT,18.391.171/0001-54,GÁVEA MACRO DÓLAR FUNDO DE INVESTIMENTO EM COT...
5572334,FI,18.762.833/0001-55,BTGP ACCESS GÁVEA MACRO DÓLAR FUNDO DE INVESTI...,2015-08-31,Cotas de Fundos,Fundo de Investimento e de Cotas,N,NaN,NaN,NaN,NaN,NaN,3647.200000,587670.37,NaN,NaT,18.391.171/0001-54,GÁVEA MACRO DÓLAR FUNDO DE INVESTIMENTO EM COT...
5579768,FI,18.936.155/0001-08,MOXUARA FUNDO DE INVESTIMENTO MULTIMERCADO - C...,2015-08-31,Cotas de Fundos,Fundo de Investimento e de Cotas,S,NaN,NaN,NaN,NaN,NaN,17820.982007,2871475.78,NaN,NaT,18.391.171/0001-54,GÁVEA MACRO DÓLAR FUNDO DE INVESTIMENTO EM COT...


In [50]:
dfBLC2[(dfBLC2['CNPJ_FUNDO_COTA'] == '08.543.320/0001-08') & 
       (dfBLC2['DT_COMPTC'] == '2015-08-31')]

,TP_FUNDO,CNPJ_FUNDO,DENOM_SOCIAL,DT_COMPTC,TP_APLIC,TP_ATIVO,EMISSOR_LIGADO,TP_NEGOC,QT_VENDA_NEGOC,VL_VENDA_NEGOC,QT_AQUIS_NEGOC,VL_AQUIS_NEGOC,QT_POS_FINAL,VL_MERC_POS_FINAL,VL_CUSTO_POS_FINAL,DT_CONFID_APLIC,CNPJ_FUNDO_COTA,NM_FUNDO_COTA


In [49]:
dfBLC2.iloc[5101000]

TP_FUNDO                                                             FI
CNPJ_FUNDO                                           08.543.320/0001-08
DENOM_SOCIAL          KRANE MULTIMERCADO FUNDO DE INV EM COTAS DE FU...
DT_COMPTC                                           2015-02-28 00:00:00
TP_APLIC                                                Cotas de Fundos
TP_ATIVO                               Fundo de Investimento e de Cotas
EMISSOR_LIGADO                                                        N
TP_NEGOC                                                            NaN
QT_VENDA_NEGOC                                                        0
VL_VENDA_NEGOC                                                        0
QT_AQUIS_NEGOC                                                        0
VL_AQUIS_NEGOC                                                        0
QT_POS_FINAL                                                     288109
VL_MERC_POS_FINAL                                           3.41

In [47]:
dfBLC2[(dfBLC2['CNPJ_FUNDO'] == '17.419.784/0001-90') & 
       (dfBLC2['DT_COMPTC'] == '2015-08-31')]

,TP_FUNDO,CNPJ_FUNDO,DENOM_SOCIAL,DT_COMPTC,TP_APLIC,TP_ATIVO,EMISSOR_LIGADO,TP_NEGOC,QT_VENDA_NEGOC,VL_VENDA_NEGOC,QT_AQUIS_NEGOC,VL_AQUIS_NEGOC,QT_POS_FINAL,VL_MERC_POS_FINAL,VL_CUSTO_POS_FINAL,DT_CONFID_APLIC,CNPJ_FUNDO_COTA,NM_FUNDO_COTA
5506316,FI,17.419.784/0001-90,VÉRTICE FOF MACRO MULTIMERCADO FUNDO DE INVEST...,2015-08-31,Cotas de Fundos,Fundo de Investimento e de Cotas,N,NaN,0.0,0.0,0.0,0.0,7.896847e+06,1.341635e+08,NaN,NaT,12.984.686/0001-81,VÉRTICE VTR FUNDO DE INVESTIMENTO EM COTAS DE ...
5506317,FI,17.419.784/0001-90,VÉRTICE FOF MACRO MULTIMERCADO FUNDO DE INVEST...,2015-08-31,Cotas de Fundos,Fundo de Investimento e de Cotas,N,NaN,0.0,0.0,0.0,0.0,9.471397e+06,1.450395e+08,NaN,NaT,14.096.013/0001-10,VÉRTICE JGP STRATEGY - FUNDO DE INVESTIMENTO E...
5506318,FI,17.419.784/0001-90,VÉRTICE FOF MACRO MULTIMERCADO FUNDO DE INVEST...,2015-08-31,Cotas de Fundos,Fundo de Investimento e de Cotas,N,NaN,0.0,0.0,0.0,0.0,3.305722e+05,4.981045e+07,NaN,NaT,15.603.942/0001-31,IBIUNA HEDGE STR FUNDO DE INVESTIMENTO EM COTA...
5506319,FI,17.419.784/0001-90,VÉRTICE FOF MACRO MULTIMERCADO FUNDO DE INVEST...,2015-08-31,Cotas de Fundos,Fundo de Investimento e de Cotas,N,NaN,0.0,0.0,0.0,0.0,1.086356e+08,1.383821e+08,NaN,NaT,18.489.908/0001-76,GROWLER FUNDO DE INVESTIMENTO EM COTAS DE FUND...
5506320,FI,17.419.784/0001-90,VÉRTICE FOF MACRO MULTIMERCADO FUNDO DE INVEST...,2015-08-31,Cotas de Fundos,Fundo de Investimento e de Cotas,N,NaN,0.0,0.0,0.0,0.0,3.933174e+07,7.143956e+07,NaN,NaT,20.270.180/0001-39,KAPITALO ZETA A FUNDO DE INVESTIMENTO EM COTAS...
5506321,FI,17.419.784/0001-90,VÉRTICE FOF MACRO MULTIMERCADO FUNDO DE INVEST...,2015-08-31,Cotas de Fundos,Fundo de Investimento e de Cotas,N,NaN,0.0,0.0,0.0,0.0,5.721093e+05,6.744471e+07,NaN,NaT,20.394.294/0001-90,SAFRA GALILEO A FUNDO DE INVESTIMENTO EM COTAS...
5506322,FI,17.419.784/0001-90,VÉRTICE FOF MACRO MULTIMERCADO FUNDO DE INVEST...,2015-08-31,Cotas de Fundos,Fundo de Investimento e de Cotas,N,NaN,0.0,0.0,0.0,0.0,1.033709e+06,1.193052e+08,NaN,NaT,21.144.556/0001-21,SAFRA GALILEO B FUNDO DE INVESTIMENTO EM COTAS...
5506323,FI,17.419.784/0001-90,VÉRTICE FOF MACRO MULTIMERCADO FUNDO DE INVEST...,2015-08-31,Cotas de Fundos,Fundo de Investimento e de Cotas,N,NaN,0.0,0.0,0.0,0.0,6.077070e+06,6.413303e+07,NaN,NaT,21.407.105/0001-30,ITAÚ TOP DI II RENDA FIXA REFERENCIADO FICFI


In [60]:
dfBLC2[(dfBLC2['CNPJ_FUNDO'] == '10.347.249/0001-21') & 
       (dfBLC2['DT_COMPTC'] == '2015-08-31')]

,TP_FUNDO,CNPJ_FUNDO,DENOM_SOCIAL,DT_COMPTC,TP_APLIC,TP_ATIVO,EMISSOR_LIGADO,TP_NEGOC,QT_VENDA_NEGOC,VL_VENDA_NEGOC,QT_AQUIS_NEGOC,VL_AQUIS_NEGOC,QT_POS_FINAL,VL_MERC_POS_FINAL,VL_CUSTO_POS_FINAL,DT_CONFID_APLIC,CNPJ_FUNDO_COTA,NM_FUNDO_COTA


In [54]:
dfBLC2[(dfBLC2['CNPJ_FUNDO_COTA'] == '17.340.475/0001-20') & 
       (dfBLC2['DT_COMPTC'] == '2015-08-31')]

,TP_FUNDO,CNPJ_FUNDO,DENOM_SOCIAL,DT_COMPTC,TP_APLIC,TP_ATIVO,EMISSOR_LIGADO,TP_NEGOC,QT_VENDA_NEGOC,VL_VENDA_NEGOC,QT_AQUIS_NEGOC,VL_AQUIS_NEGOC,QT_POS_FINAL,VL_MERC_POS_FINAL,VL_CUSTO_POS_FINAL,DT_CONFID_APLIC,CNPJ_FUNDO_COTA,NM_FUNDO_COTA
5180723,FI,09.536.078/0001-08,YANKEE FUNDO DE INVESTIMENTO MULTIMERCADO CRED...,2015-08-31,Cotas de Fundos,Fundo de Investimento e de Cotas,N,NaN,0.0,0.0,0.0,0.0,498386.53669,85913825.62,NaN,2015-11-29,17.340.475/0001-20,FEEDER GAVEA MACRO DOLAR FUNDO DE INVESTIMENTO...


In [37]:
dfBLC2[(dfBLC2['CNPJ_FUNDO_COTA'] == '18.391.163/0001-08') & 
       (dfBLC2['DT_COMPTC'] == '2015-08-31')]

,TP_FUNDO,CNPJ_FUNDO,DENOM_SOCIAL,DT_COMPTC,TP_APLIC,TP_ATIVO,EMISSOR_LIGADO,TP_NEGOC,QT_VENDA_NEGOC,VL_VENDA_NEGOC,QT_AQUIS_NEGOC,VL_AQUIS_NEGOC,QT_POS_FINAL,VL_MERC_POS_FINAL,VL_CUSTO_POS_FINAL,DT_CONFID_APLIC,CNPJ_FUNDO_COTA,NM_FUNDO_COTA
5163723,FI,09.289.128/0001-91,CSHG PORTO CERVO II FC FI MULTIMERCADO - CRÉDI...,2015-08-31,Cotas de Fundos,Fundo de Investimento e de Cotas,N,NaN,0.0,0.0,0.0,0.0,100204.264100,1.736445e+07,NaN,2015-11-29,18.391.163/0001-08,APOENA MACRO DÓLAR FUNDO DE INVESTIMENTO EM CO...
5289848,FI,11.960.140/0001-28,GAJI MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO...,2015-08-31,Cotas de Fundos,Fundo de Investimento e de Cotas,N,NaN,0.0,0.0,0.0,0.0,385005.179010,6.671775e+07,NaN,NaT,18.391.163/0001-08,APOENA MACRO DÓLAR FUNDO DE INVESTIMENTO EM CO...
5290409,FI,11.960.495/0001-17,EJAS MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO...,2015-08-31,Cotas de Fundos,Fundo de Investimento e de Cotas,N,NaN,0.0,0.0,0.0,0.0,180850.252380,3.133964e+07,NaN,NaT,18.391.163/0001-08,APOENA MACRO DÓLAR FUNDO DE INVESTIMENTO EM CO...
5296423,FI,12.065.963/0001-52,SANTANDER FI PB GAGARIN MULTIMERCADO CREDITO P...,2015-08-31,Cotas de Fundos,Fundo de Investimento e de Cotas,N,NaN,NaN,NaN,NaN,NaN,147852.072681,2.562136e+07,NaN,2015-11-29,18.391.163/0001-08,APOENA MACRO DÓLAR FUNDO DE INVESTIMENTO EM CO...
5396272,FI,14.415.875/0001-69,PALMER MULTIMERCADO CRÉDITO PRIVADO INVESTIMEN...,2015-08-31,Cotas de Fundos,Fundo de Investimento e de Cotas,N,NaN,0.0,0.0,0.0,0.0,588492.682778,1.019802e+08,NaN,NaT,18.391.163/0001-08,APOENA MACRO DÓLAR FUNDO DE INVESTIMENTO EM CO...
5550698,FI,18.247.733/0001-90,FUNDO DE INVESTIMENTO MULTIMERCADO CRÉDITO PRI...,2015-08-31,Cotas de Fundos,Fundo de Investimento e de Cotas,N,NaN,NaN,NaN,NaN,NaN,3225.740000,5.589910e+05,NaN,NaT,18.391.163/0001-08,APOENA MACRO DÓLAR FUNDO DE INVESTIMENTO EM CO...
5607042,FI,19.765.157/0001-36,CRUISE PLUS FICFI MULTIMERCADO CREDITO PRIVADO...,2015-08-31,Cotas de Fundos,Fundo de Investimento e de Cotas,N,NaN,NaN,NaN,NaN,NaN,1465.265735,2.539167e+05,NaN,NaT,18.391.163/0001-08,APOENA MACRO DÓLAR FUNDO DE INVESTIMENTO EM CO...
